In [1]:
import csv
import json
import random
from enum import Enum

# 1. Player Class
class Player:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.form = PlayerForm.NORMAL  # Starting state for player form

    def __str__(self):
        return self.name

    def win(self):
        self.score += 3  # Win = 3 points
        self.update_form(True)

    def lose(self):
        self.update_form(False)

    def draw(self):
        self.score += 1  # Draw = 1 point

    def update_form(self, won_last_game):
        """Updates player's form based on win/lose/draw."""
        if won_last_game:
            if self.form == PlayerForm.NORMAL:
                self.form = PlayerForm.ON_FORM
            elif self.form == PlayerForm.OFF_FORM:
                self.form = PlayerForm.NORMAL
        else:
            if self.form == PlayerForm.NORMAL:
                self.form = PlayerForm.OFF_FORM
            elif self.form == PlayerForm.ON_FORM:
                self.form = PlayerForm.NORMAL

# 2. Match Class
class Match:
    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self.result = None  # None for not decided, 'draw', 'player1', 'player2'

    def play(self):
        """Simulate a match and determine the winner."""
        print(f"Match between {self.player1} and {self.player2}")
        outcome = random.choice([None, 'player1', 'player2', 'draw'])  # Random outcome for simplicity
        self.result = outcome
        if outcome == 'player1':
            self.player1.win()
            self.player2.lose()
        elif outcome == 'player2':
            self.player2.win()
            self.player1.lose()
        elif outcome == 'draw':
            self.player1.draw()
            self.player2.draw()
        self.log_match()

    def log_match(self):
        """Log the match result into a CSV file."""
        with open('tournament_log.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([self.player1.name, self.player2.name, self.result])

# 3. Tournament Class with Strategy Pattern for tournament type
class Tournament:
    def __init__(self, name, tournament_type):
        self.name = name
        self.tournament_type = tournament_type
        self.players = []
        self.matches = []
        self.rounds = 0
        self.completed = False

    def add_player(self, player):
        self.players.append(player)

    def start(self):
        print(f"Starting {self.name} tournament...")
        self.matches = self.tournament_type.generate_matches(self.players)
        for match in self.matches:
            match.play()
        self.completed = True
        self.generate_awards()

    def generate_awards(self):
        top_scorer = max(self.players, key=lambda p: p.score)
        best_player = max(self.players, key=lambda p: p.form.value)
        print(f"Awards for {self.name}:")
        print(f"Top Scorer: {top_scorer.name} with {top_scorer.score} points")
        print(f"Best Player: {best_player.name} with form: {best_player.form.name}")
        self.export_results()

    def export_results(self):
        """Export the tournament table as a JSON file."""
        tournament_data = {
            "name": self.name,
            "players": [{"name": player.name, "score": player.score, "form": player.form.name} for player in self.players],
            "matches": [{"player1": match.player1.name, "player2": match.player2.name, "result": match.result} for match in self.matches]
        }
        with open(f"{self.name}_results.json", 'w') as f:
            json.dump(tournament_data, f, indent=4)

# 4. Referee Class
class Referee:
    def __init__(self, name):
        self.name = name

    def officiate(self, match):
        """Referee oversees the match, in this case it's a simulation."""
        print(f"{self.name} is officiating the match between {match.player1} and {match.player2}")

# 5. Tournament Types using Strategy Pattern
class TournamentType(Enum):
    ROUND_ROBIN = 1
    KNOCKOUT = 2

    def generate_matches(self, players):
        """Generate matches based on the tournament type."""
        if self == TournamentType.ROUND_ROBIN:
            return self.generate_round_robin(players)
        elif self == TournamentType.KNOCKOUT:
            return self.generate_knockout(players)

    @staticmethod
    def generate_round_robin(players):
        """Generate Round Robin matches."""
        matches = []
        for i in range(len(players)):
            for j in range(i + 1, len(players)):
                matches.append(Match(players[i], players[j]))
        return matches

    @staticmethod
    def generate_knockout(players):
        """Generate Knockout matches (simplified)."""
        random.shuffle(players)
        matches = []
        while len(players) > 1:
            player1 = players.pop()
            player2 = players.pop()
            matches.append(Match(player1, player2))
        return matches

# 6. Player Form using State Pattern
class PlayerForm(Enum):
    NORMAL = 1
    ON_FORM = 2
    OFF_FORM = 3

# 7. CLI for Admin to manage tournament
def admin_cli():
    print("Welcome to the Tournament Manager")
    tournament_name = input("Enter Tournament Name: ")
    tournament_type_input = input("Enter Tournament Type (1 for Round Robin, 2 for Knockout): ")
    tournament_type = TournamentType.ROUND_ROBIN if tournament_type_input == '1' else TournamentType.KNOCKOUT

    tournament = Tournament(tournament_name, tournament_type)

    while True:
        action = input("What would you like to do? (1 to add player, 2 to start tournament, 3 to exit): ")
        if action == '1':
            player_name = input("Enter Player Name: ")
            player = Player(player_name)
            tournament.add_player(player)
            print(f"Added player {player_name}")
        elif action == '2':
            tournament.start()
            break
        elif action == '3':
            print("Exiting...")
            break
        else:
            print("Invalid option")

if __name__ == "__main__":
    admin_cli()


Welcome to the Tournament Manager
Added player Ronaldo
Added player Messi
Starting LaLiga tournament...
Match between Ronaldo and Messi
Awards for LaLiga:
Top Scorer: Messi with 3 points
Best Player: Ronaldo with form: OFF_FORM
